# STATISTIQUES DESCRIPTIVES

*Source: cette série d'exercices s'inpire très largement des tutoriels de Simon Ejdemyr: https://sejdemyr.github.io/r-tutorials/statistics/*

Pour produire quelques statistiques descriptives et graphiques, commençons par ouvrir la base de données : *anescum_small_v12.dta* et sélectionner quelques variables qui nous intéressent : 

*(notons que les données sont sous format Stata dta mais que R n'a aucune difficulté à les ouvrir avec le package **foreign** et la commande **read.dta**, pourvu qu'elles soient sauvées par une version Stata 14 ou antérieure)*

In [ ]:
#ouverture des données:
library(foreign) #pour ouvrir les données dta de version 14 ou inférieure
anes <- read.dta("TD3_anescum_small_v12.dta")

In [ ]:
#on garde seulement quelques variables et on les renomme pour plus de clarté:
library(plyr)
library(dplyr)
anes <- anes %>% 
        select(year = VCF0004,
               age = VCF0101,
               gender = VCF0104,
               race = VCF0106,
               edu = VCF0140,
               south = VCF0113,
               income = VCF0114,
               partyid = VCF0301,
               interest = VCF0310,
               govtrust = VCF0604,
               abortion = VCF0838,
               demtherm = VCF0218)
#on regarde les 6 premières observations:
head(anes)

Codebook de ces données : TD3_codebook_anescum_small_v12.txt

# 1. Statistiques descriptives univariées

Pour analyser des données il est important d'avoir une idée de leur distribution. Onutilisera généralement le package **ggplot2** pour les graphiques. 

Le type de statistique diffère selon si la variable est discrète ou continue. 

### 1.1 Variables discrètes

On s'intéresse à l'opinion politique des américains: variable *partyid*. Cette variable est catégorielle/discrète: 

In [ ]:
levels(anes$partyid)
table(anes$partyid)

On la recode en 3 catégories: démocrate, indépendant, républicain:

In [ ]:
anes$partyid3 <- anes$partyid #dupliquer

levels(anes$partyid3)<-c(NA,
                         "Democrat","Democrat","Democrat",
                         "Independent",
                         "Republican","Republican","Republican") #redéfinir catégories

table(anes$partyid3) #nombre
prop.table(table(anes$partyid3)) #proportion

On peut faire un examen rapide de la distribution de la variable par un simple **barplot()** avec les proprtions :

In [ ]:
barplot(prop.table(table(anes$partyid3)))

### 1.2 Variables continues
Beaucoup de commandes nous aident à connnaître la distribution: **mean()**, **median()**, **min()**, **max()**, **sd()**.

*NOTE: ne pas oublier d'ajouter **na.rm = T** si on veut ignorer les observations variables manquantes.*

In [ ]:
mean(anes$demtherm)
mean(anes$demtherm, na.rm=T)

median(anes$demtherm, na.rm = T)
min(anes$demtherm, na.rm = T)
max(anes$demtherm, na.rm = T)
sd(anes$demtherm, na.rm=T)

On retrouve beaucup de ces commandes en une: **summary()**.

In [ ]:
summary(anes$demtherm)

Pour obtenir un quantile spécifique: 

In [ ]:
quantile(anes$demtherm, probs = c(0.1, 0.5, 0.9), na.rm = T)

Ou de manière incrémentale:

In [ ]:
quantile(anes$demtherm, probs = seq(0, 1, by = 0.05), na.rm = T)

On peut faire un histogramme rapide: 

In [ ]:
hist(anes$demtherm)

### Exercice 1

Etudier la distribution de la variable *interest* (une variable catégorielle qui mesure 'intérêt porté à la politique), c'est-à-dire : 
- le nombre 
- et la proportion d'observations pour chaque catégorie de la variable.

In [ ]:
class(anes$interest)
levels(anes$interest)<-c(NA,"Not much","Somewhat","Very",NA)
table(anes$interest, exclude=NULL)
t<-prop.table(table(anes$interest))
t
barplot(t)

Trouver les statistiques descriptives d'une variable numérique de votre choix: moyenne, médiane, écart type, minimum, maximum, 7 et 45ème percentile. 

In [ ]:
head(anes)
class(anes)
class(anes$govtrust)
class(anes$age) #celle-ci est numérique

summary(anes$age)
sd(anes$age, na.rm=T)
quantile(anes$age, probs=c(0.07,0.45), na.rm=T)

# 2. Statistiques descriptives bivariées
Il est encore plus intéressant d'étudier la relation entre deux variables et plus. 

Les statistiques bivariées prennent la forme de moyennes conditionnelles, de tableaux croisés, de graphiques. 

Encore une fois, le typde de statistique dépend de la nature discrète ou continue des données. 

### 2.1 Deux variables discrètes/catégorielles
Par exemple, quelle est la relation entre genre et parti politique? On veut un tableau croisé. On peut l'obtenir en fréquence (**table()**) ou en proportion (**prop.table()**). 

In [ ]:
#on examine la variable genre 
levels(anes$gender) 

#et on la recode en simplifiant:
levels(anes$gender)<- c(NA, "Male", "Female")
levels(anes$gender) 

#tablau simple
table(anes$gender)

In [ ]:
#tableau croisé:
table(anes$gender, anes$partyid3)   #option1
with(anes, table(gender, partyid3)) #option2

In [ ]:
#proportions, somme 100 en ligne
t <- with(anes, table(gender, partyid3))
prop.table(t, margin = 1)

NOTE 1: Pourquoi 'margin=1' ? Comparer **prop.table(t, margin = 1)** avec ce qui suit et expliquer la différence.

In [ ]:
prop.table(t)          
prop.table(t,margin=2) 

NOTE 2: Fonction **ddply**
On peut retrouver ces statistiques avec une fonction très utile du package **plyr**. Cette fonction est beaucoup plus utilisée que **prop.table()**. 
- argument 1 : data frame
- argument 2 : variable indépendante, à partir de laquelle on souhaite étudier les sous-groupes
- argument 3 : ici summarise car on veut une stat des
- argument 4 : une foncion, ici la moyenne de la variable continue par groupe

Exemple :

In [ ]:
anes$democrat <- anes$partyid3
levels(anes$democrat) <- c(1, 0, 0)
anes$democrat <- as.numeric(as.character(anes$democrat)) #on transforme la variable catégorielle en variable numérique

ddply(anes, .(gender), summarise,
         mean = mean(democrat, na.rm = T),
         median = median(democrat, na.rm = T))

### 2.2 Une variable discrète et une variable continue
On cherche alors les moyennes conditionnelles en utilisant encore la fonction **ddply()**. 

Par exempe, si je cherche la moyenne conditionnelle de la sympathie pour les démocrates (*demtherm*, variable continue) le long de la distribution du revenu en quintiles (*income*, variable catégorielle), j'écris:

In [ ]:
ddply(anes, .(income), summarise, mean = mean(demtherm, na.rm = T),
                                  median = median(demtherm, na.rm = T))

### 2.3 Deux variables continues
Ce type de relation est davantage étudié graphiquement. Il exitse toute de même la corrélation de Pearson entre deux variables, par exemple l'âge (*age*) et la symaptahie pour les démocrates (*demtherm*) :

In [ ]:
cor(anes$age, anes$demtherm, use = "complete.obs")

### Exercice 2

Créer un tableau croisé entre *south* (Sud/Nord) et la variable *partyid3* (parti démocrate, répubicain ou indépendant), séparément pour 1952 et 2008 (variable *year*). Analyser l'évolution.

Utiliser la variable *democrat* crée ci-dessus et la fonction **ddply()** pour trouver la proportion de démocrates, et l'écart type de *democrat*, par région (sud/non-sud) et année (toutes les valeurs, pas seulement 1952 et 2008).  

# 3. Réaliser des graphiques avec ggplot

**ggplot2** est le package performant de R pour faire tout type de graphique. 

L'idée ici n'est pas de faire un compte rendu exhaustif de **ggplot2** mais de donner quelques exemples. 

Un lien pour davantage d'exemples: http://www.cookbook-r.com/Graphs/

### 3.1 Exemple 1 : Proprtion de démocrates par genre et année

In [ ]:
#On calcule d'abord ces proportions: 
t<-ddply(anes, .(gender,year), summarise, mean=mean(democrat, na.rm = T))
tail(t)

In [ ]:
#On crée un graphique
library(ggplot2)
ggplot(t, aes(x = year, y = mean, color = gender)) +
    geom_point() 

In [ ]:
#on relie les points
library(ggplot2)
ggplot(t, aes(x = year, y = mean, color = gender)) +
    geom_point() + geom_line()

In [ ]:
#on ajoute une légende à y et on change le fond
ggplot(t, aes(x = year, y = mean, color = gender)) +
    geom_point() +
    geom_line() +
    xlab("") +
    ylab("Proportion de Démocrates") +
    theme_bw()

### 3.2 Exemple 2 : Thermomètre démocrate

In [ ]:
#Densité du thermoùètre par genre:
ggplot(anes, aes(demtherm, fill = gender)) +
    geom_density()

In [ ]:
#transparence 0.3, fenêtre plus large (adjust), pas de contour 
ggplot(anes, aes(demtherm, fill = gender)) +
    geom_density(alpha = 0.3, adjust = 2, colour = NA)

In [ ]:
#label des axes, changement de couleurs et de fond
ggplot(anes, aes(demtherm, fill = gender)) +
    geom_density(alpha = 0.3, adjust = 2, colour = NA) +
    xlab("Democratic Thermometer") +
    ylab("Density") +
    scale_fill_manual(values = c("black", "darkblue")) +
    theme_bw()

In [ ]:
#une combinsaison de graphiques avec facet_wrap:
years <- seq(1980, 2008, by = 4)
anes_sub <- subset(anes, year %in% years)

ggplot(anes_sub, aes(demtherm, fill = gender)) +
    geom_density(alpha = 0.3, adjust = 1.8, colour = NA) +
    xlab("Democratic Thermometer") +
    ylab("Density") +
    scale_fill_manual(values = c("black", "darkblue")) +
    theme_bw() +
    facet_wrap(~year)

### 3.3 Sauvegarder les graphiques

NOTE: Pour sauvegarder et insérer dans des documents, le mieux est le format pdf. 
- On sauvegarde le graphique comme objet, 
- et on l'encadre entre **pdf()** et **dev.off()**

L'image sera dans le répertoire de travail.

Exemple:

g1<- ggplot(anes_sub, aes(demtherm, fill = gender)) +
    geom_density(alpha = 0.3, adjust = 1.8, colour = NA) +
    xlab("Democratic Thermometer") +
    ylab("Density") +
    scale_fill_manual(values = c("black", "darkblue")) +
    theme_bw() +
    facet_wrap(~year)
    
pdf(file = "TD1_density.pdf", height = 6, width = 8)
    
p

dev.off()

### Exercice 3

Représenter le nuage de points (scatter plot) du thermomètre démocrate (*demtherm*) en fonction de l'âge (*age*) pour 1978. 

Refaire le même nuage de points pour les années 1978, 1886, 1994 et 1998. Utiliser **facet_wrap()**.

Dans l'exercice 2 nous avons calculé la proportion de démocrates par région (*south*) et année (*year*). Utiliser **ggplot** pour représenter ces estimations. Le temps est en abscisse, l'appartenance au parti démocrate en ordonnée. Relier les points par des lignes. Conclure. 